# Final Project: Tesla & GameStop — Stock vs Revenue Dashboard
**Run top-to-bottom. After each Question, take a screenshot of the code + output.**

If any library is missing, run the install cell below.

In [ ]:
# If needed, uncomment to install
# !pip install --quiet yfinance pandas requests beautifulsoup4 plotly kaleido lxml html5lib


In [ ]:
import warnings, re, os
warnings.filterwarnings('ignore')

import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go


In [ ]:
def _clean_revenue_table(df: pd.DataFrame) -> pd.DataFrame:
    """Take a 2-col (Date, Revenue) table, clean currency formatting, and return a sorted DataFrame."""
    df = df.iloc[:, :2].copy()
    df.columns = ['date', 'revenue']
    df = df.dropna()
    df['revenue'] = (df['revenue'].astype(str)
                     .str.replace(r'\$|,', '', regex=True)
                     .str.strip()
                     .replace({'': None}))
    df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
    df = df.dropna(subset=['revenue'])
    df['date'] = pd.to_datetime(df['date'])
    return df.sort_values('date').reset_index(drop=True)


In [ ]:
def make_graph(stock_data: pd.DataFrame, revenue_data: pd.DataFrame, title: str = ""):
    s = stock_data.copy()
    if 'Date' in s.columns:
        s['Date'] = pd.to_datetime(s['Date'])
    else:
        s = s.reset_index().rename(columns={'index': 'Date'})
        s['Date'] = pd.to_datetime(s['Date'])

    r = revenue_data.copy()
    if 'Date' in r.columns:
        r['Date'] = pd.to_datetime(r['Date'])
        r = r.rename(columns={'Revenue':'revenue'})
    else:
        r = r.rename(columns={'date':'Date'})

    start = max(s['Date'].min(), r['Date'].min())
    end   = min(s['Date'].max(), r['Date'].max())
    s = s[(s['Date'] >= start) & (s['Date'] <= end)]
    r = r[(r['Date'] >= start) & (r['Date'] <= end)]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=s['Date'], y=s['Close'], name='Close Price', mode='lines'))
    fig.add_trace(go.Bar(x=r['Date'], y=r['revenue'], name='Revenue', yaxis='y2', opacity=0.5))

    fig.update_layout(
        title=title or 'Stock Price vs Revenue',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Close Price (USD)'),
        yaxis2=dict(title='Revenue (USD)', overlaying='y', side='right'),
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
        bargap=0
    )
    fig.show()
    # Optional: also save images if kaleido is installed
    try:
        safe_title = re.sub(r'[^a-zA-Z0-9_-]+', '_', title or 'chart')
        fig.write_image(f"{safe_title}.png", width=1200, height=700, scale=2)
        print(f"Saved {safe_title}.png")
    except Exception as e:
        print('(Note) Could not save PNG (kaleido missing?):', e)


## Question 1 — Use yfinance to Extract Tesla Stock Data

In [ ]:
tesla = yf.Ticker('TSLA')
tesla_data = tesla.history(period='max')
tesla_data = tesla_data.reset_index()
tesla_data.to_csv('tesla_data.csv', index=False)
tesla_data.head()


## Question 2 — Use Webscraping to Extract Tesla Revenue Data

In [ ]:
tesla_rev_url = 'https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue'
html = requests.get(tesla_rev_url, headers={'User-Agent':'Mozilla/5.0'}).text
soup = BeautifulSoup(html, 'html.parser')
all_tables = pd.read_html(str(soup))
candidates = []
for t in all_tables:
    if t.shape[1] >= 2 and ('Revenue' in ' '.join(t.columns.astype(str)) or t.iloc[:,1].astype(str).str.contains(r'\$|,').any()):
        candidates.append(t.iloc[:, :2])
if not candidates:
    raise RuntimeError('No revenue table found.')

_tesla_revenue = _clean_revenue_table(candidates[0])
tesla_revenue = _tesla_revenue.rename(columns={'date':'Date','revenue':'Revenue'})
tesla_revenue.tail()


## Question 3 — Use yfinance to Extract GameStop Stock Data

In [ ]:
gme = yf.Ticker('GME')
gme_data = gme.history(period='max')
gme_data = gme_data.reset_index()
gme_data.to_csv('gme_data.csv', index=False)
gme_data.head()


## Question 4 — Use Webscraping to Extract GameStop Revenue Data

In [ ]:
gme_rev_url = 'https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue'
html = requests.get(gme_rev_url, headers={'User-Agent':'Mozilla/5.0'}).text
soup = BeautifulSoup(html, 'html.parser')
all_tables = pd.read_html(str(soup))
candidates = []
for t in all_tables:
    if t.shape[1] >= 2 and ('Revenue' in ' '.join(t.columns.astype(str)) or t.iloc[:,1].astype(str).str.contains(r'\$|,').any()):
        candidates.append(t.iloc[:, :2])
if not candidates:
    raise RuntimeError('No revenue table found.')
_gme_revenue = _clean_revenue_table(candidates[0])
gme_revenue = _gme_revenue.rename(columns={'date':'Date','revenue':'Revenue'})
gme_revenue.tail()


## Question 5 — Plot Tesla Stock Graph

In [ ]:
make_graph(tesla_data[['Date','Close']], tesla_revenue, title='Tesla_TSLA_Close_vs_Quarterly_Revenue')

## Question 6 — Plot GameStop Stock Graph

In [ ]:
make_graph(gme_data[['Date','Close']], gme_revenue, title='GameStop_GME_Close_vs_Quarterly_Revenue')

## Question 7 — Share your Assignment Notebook
Upload this notebook to GitHub/Watson Studio and share the URL.

In [ ]:
# Helper: list any files generated (CSVs, PNGs if kaleido installed)
import os
[p for p in os.listdir('.') if p.endswith(('.csv','.png','.html'))]